In [1]:
import warnings
warnings.filterwarnings('ignore')

from tensorflow.python.keras.models import Model

from tensorflow.python.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.layers import Input, Dense
from tensorflow.python.keras.layers import Dense, Dropout, Flatten


import matplotlib.pyplot as plt
import numpy as np
import os
import torch

import tensorflow as tf
import cv2
from tensorflow.keras.preprocessing import image_dataset_from_directory


from matplotlib import pyplot as plt

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

Using cuda device


In [2]:
train_generator = ImageDataGenerator(rotation_range=90, 
                                     brightness_range=[0.1, 0.7],
                                     width_shift_range=0.5, 
                                     height_shift_range=0.5,
                                     horizontal_flip=True, 
                                     vertical_flip=True,
                                     validation_split=0.15,
                                     preprocessing_function=preprocess_input) # VGG16 preprocessing

test_generator = ImageDataGenerator(preprocessing_function=preprocess_input) # VGG16 preprocessing

In [3]:
train_data_dir = 'archive (2)/Animal Image Dataset/train'
test_data_dir = 'archive (2)/Animal Image Dataset/test'

In [4]:
BATCH_SIZE=64
traingen = train_generator.flow_from_directory(train_data_dir,
                                               target_size=(128, 128),
                                               subset='training',
                                               batch_size=BATCH_SIZE, 
                                               shuffle=True,
                                               seed=42)

validgen = train_generator.flow_from_directory(train_data_dir,
                                               target_size=(128, 128),
                                               subset='validation',
                                               batch_size=BATCH_SIZE,
                                               shuffle=True,
                                               seed=42)

testgen = test_generator.flow_from_directory(test_data_dir,
                                             target_size=(128, 128),
                                             batch_size=1,
                                             shuffle=False,
                                             seed=42)

Found 10954 images belonging to 12 classes.
Found 1927 images belonging to 12 classes.
Found 4296 images belonging to 12 classes.


In [5]:
def create_model(input_shape, n_classes, optimizer='rmsprop', fine_tune=0):

    
    # Pretrained convolutional layers are loaded using the Imagenet weights.
    # Include_top is set to False, in order to exclude the model's fully-connected layers.
    conv_base = VGG16(include_top=False,
                     weights='imagenet', 
                     input_shape=input_shape)
    
    # Defines how many layers to freeze during training.
    # Layers in the convolutional base are switched from trainable to non-trainable
    # depending on the size of the fine-tuning parameter.
    if fine_tune > 0:
        for layer in conv_base.layers[:-fine_tune]:
            layer.trainable = False
    else:
        for layer in conv_base.layers:
            layer.trainable = False

    # Create a new 'top' of the model (i.e. fully-connected layers).
    # This is 'bootstrapping' a new top_model onto the pretrained layers.
    top_model = conv_base.output
    top_model = Flatten(name="flatten")(top_model)
    output_layer = Dense(n_classes, activation='softmax')(top_model)
    
    # Group the convolutional base and new fully-connected layers into a Model object.
    model = Model(inputs=conv_base.input, outputs=output_layer)

    # Compiles the model for training.
    model.compile(optimizer=optimizer, 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [8]:
input_shape = (128, 128, 3)
optim_1 = tf.keras.optimizers.Adam(learning_rate=0.001)
n_classes=12

n_steps = traingen.samples // BATCH_SIZE
n_val_steps = validgen.samples // BATCH_SIZE
n_epochs = 5

vgg_model = create_model(input_shape, n_classes, optim_1, fine_tune=0)

In [9]:
vgg_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0   

In [10]:
vgg_history = vgg_model.fit(traingen,
                            batch_size=BATCH_SIZE,
                            epochs=n_epochs,
                            validation_data=validgen,
                            steps_per_epoch=n_steps,
                            validation_steps=n_val_steps,
                            verbose=1)

Epoch 1/5
171/171 [==============================] - 688s 4s/step - loss: 10.8935 - accuracy: 0.3244 - val_loss: 8.3474 - val_accuracy: 0.4615
Epoch 2/5
171/171 [==============================] - 667s 4s/step - loss: 7.3676 - accuracy: 0.5045 - val_loss: 8.5501 - val_accuracy: 0.4823
Epoch 3/5
171/171 [==============================] - 676s 4s/step - loss: 7.5473 - accuracy: 0.5187 - val_loss: 8.2213 - val_accuracy: 0.5318
Epoch 4/5
171/171 [==============================] - 674s 4s/step - loss: 7.4623 - accuracy: 0.5397 - val_loss: 8.5195 - val_accuracy: 0.5276
Epoch 5/5
171/171 [==============================] - 671s 4s/step - loss: 7.4128 - accuracy: 0.5560 - val_loss: 8.6388 - val_accuracy: 0.5109


In [12]:

true_classes = testgen.classes
class_indices = traingen.class_indices
class_indices = dict((v,k) for k,v in class_indices.items())

vgg_preds = vgg_model.predict(testgen)
vgg_pred_classes = np.argmax(vgg_preds, axis=1)

In [13]:
from sklearn.metrics import accuracy_score

vgg_acc = accuracy_score(true_classes, vgg_pred_classes)
print("VGG16 Model Accuracy without Fine-Tuning: {:.2f}%".format(vgg_acc * 100))

VGG16 Model Accuracy without Fine-Tuning: 80.47%
